# How to be Choosy: Toxic Release Inventory 2023 - CA
_by Michelle Hoda Wilkerson_

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CalCoRE/choosy/main?urlpath=%2Fdoc%2Ftree%2Ftri23ca.ipynb) <- Click here to open an interactive version of this notebook!

<div class="alert alert-block alert-info"> <b>NOTE:</b> This notebook is written to illustrate the data wrangling moves described in [anonymized manuscript], using a dataset about toxic release events in California as reported by the U. S. Environmental Protection Agency in the 2023 (see <a href="#about">here</a> for more information). Some of the text below is excerpted from the manuscript. Each section below corresponds to a subheading in the manuscript. See also How to be Choosy: Billboard Hot 100. </div>

This is a Jupyter Python notebook. It mixes together Python code and text into one runnable, editable document (like a Word file or Google Doc). When you see code, you can run it by clicking the "Play" button in your interface. You can also edit it and see what happens! Often, pieces of code that appear later in the document rely on piece of code that appear earlier. So if you are a beginner, it's useful to work through every chunk of the document in order, top to bottom, before hopping around or making too many edits.

With any Jupyter notebook, we begin by importing the necessary libraries. You should run the cell below before running any other code. Here, we are using the `pandas` library, an industry standard that provides us with special methods for reading in and working with data in Python. We also import `folium`, which allows us to create maps using latitude and longitude data.

In [ ]:
import pandas # for data wrangling
import folium # for the map
import numpy as np # for calculations
import warnings # to silence warnings

# if there are more than 10 records to show in a table, only show the first and last 5
pandas.set_option('display.max_rows', 10) 
#suppress warnings
warnings.filterwarnings('ignore')

# Introduction to TRI23CA

Let's check out the dataset. In the setup code above, we loaded the `pandas` library and other libraries to help us process this data. In the code below, we read the csv file and use `pandas` to turn it into a data frame called `tri23ca`. The next line of code, which just says `tri23ca`, will give a brief summary of the contents of the dataframe so you can check and make sure it has been read and processed correctly. Since there are too many cases to list, you will see the first five rows and the last five rows of the dataset, with "..." in the middle to indicate there are more cases that are not shown.

In [ ]:
# read the contents of tri23ca.csv into a dataframe
tri23ca = pandas.read_csv("tri23ca.csv")

tri23ca # show us the contents of the new data frame

At the bottom of the output from our code above, we can see the specific dimensions of this dataset. It has 3482 cases (rows), and 122 attributes (columns). Below, we model the different wrangling strategies described in _How to be Choosy_ to reduce the size and/or complexity of the `tri23ca` dataset so that it is more appropriate for different educational applications.

Let's create a map of all the toxic release events recorded in the tri23ca dataset, using the listed latitude and longitude of each site.

In [ ]:
# Calculate the center of the map
center_lat = tri23ca['Latitude'].mean()
center_lon = tri23ca['Longitude'].mean()

# Create the Folium map
map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Add markers for each row in the DataFrame
for index, row in tri23ca.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']

    # Add the marker
    folium.Marker([lat, lon]).add_to(map)

map

# Table of Contents
* [Wrangling Too Many Cases](#Wrangling-Too-Many-Cases-)
    * [Random Selection](#Random-Selection-)
    * [Purposeful Selection by Attribute(s)](#Purposeful-Selection-by-Attribute(s)-)
    * [Building Your Own Selection Attribute](#Building-Your-Own-Selection-Attribute-)
* [Wrangling Too Many Attributes](#Wrangling-Too-Many-Attributes-)
    * [Thematic Selection](#Thematic-Selection-)
    * [Pattern-Driven Selection](#Pattern-Driven-Selection-)
    * [Question-Driven Selection](#Question-Driven-Selection-)
* [More About This Dataset](#About-the-Toxic-Release-Inventory-2023---CA-Dataset)

# Wrangling Too Many Cases <a class="anchor" id="cases"></a>

## Random Selection <a class="anchor" id="random"></a>

Random selection extracts random rows from a large dataset to create one of a more manageable size. This is the most appropriate strategy for downsizing a dataset while preserving a representative snapshot of the full phenomenon to be studied. Starting with your dataframe, you can create a random selection from that dataframe using the sample() method. Below, we select exactly 100 random cases from the `tri23ca` dataset and save them as a new reduced dataset called minitri23ca. If you run the code more than once, you will see that different cases are included in the dataset output each time.

In [ ]:
minitri23ca = tri23ca.sample(100) # put a randomly selected 5000 rows in bh100reduced
minitri23ca

A related technique is interpolated selection, or selecting every <i>n</i>th row of a data table. This might be useful when the order of the data matters (for example, if records are organized by date and you are interested in modeling patterns over time). However, we do not recommend interpolated selection unless you have a specific reason for using this method, because it can also lead to unintentionally non-random sampling.

In [ ]:
minitri23ca = tri23ca.iloc[::30, :] # put every 30th row of tri23ca in minitri23ca
minitri23ca

## Purposeful Selection by Attribute(s) <a class="anchor" id="purposeful"></a>

Purposeful selection involves reducing a dataset so that it only includes records with certain characteristics related to one or more attributes. This method is appropriate if you suspect that the majority of records in your too-large dataset are not useful or usable for your intended activity. To get all the cases in a dataframe that meet certain conditions, use the expression dataframe[condition]. Below, we want only the toxic releases that involved known carcinogens, represented in tri23ca by records where the ‘Carcinogen’ value is set to YES.

In [ ]:
carcinogens = tri23ca['Carcinogen']=="YES"  # for each song, see if the highest position was 1
minitri23ca = tri23ca[carcinogens]              # put only top song records in the reduced dataset
minitri23ca

## Building Your Own Selection Attribute <a class="anchor" id="byo"></a>

There are other ways of creating a smaller dataset based on information that is not already available in the dataset itself. These could be specific cases that you identify manually, or cases you might identify using computer code to extract some new, meaningful information from the attributes you already have. These techniques allow teachers and students the most customization, but they require careful planning to select and identify which cases should be included and to consider how those decisions will shape what analyses and claims are appropriate. 

### By Identifying Specific Indices

Sometimes, you may want to create a small dataset through manually selecting a small number of cases from a larger data corpus. This can be helpful, for example, if you would like students to work with a small set of familiar cases before diving into larger-scale analyses. The Billboard Hot 100 notebook shows an example of how you might build a list of the ID numbers or _indices_ of the cases you want to include. Below, we create a dataset by directly searching for certain words, in order to create a dataset that includes records from facilities that are directly mentioned in the fifth paragraph of [this 2023 CalMatters article](https://calmatters.org/environment/2023/12/california-hazardous-toxic-waste-mexico/). These facilities are presented as examples of some of the many California-based facilities that export their waste to recycling facilities in Mexico.

In [ ]:
# use the lines below to create a list of all the records with Facility names
# the plus sign attaches lists of records for each keyword together into one list

mentionedFacilities =  tri23ca['Facility name'].str.contains("SHERWIN") + tri23ca['Facility name'].str.contains("NAVY") + tri23ca['Facility name'].str.contains("TESLA")
minitri23ca = tri23ca[mentionedFacilities]
minitri23ca # show the full records

### Using Code to Construct a New Attribute

In other cases, you may want to use code to systematically filter certain records for deeper analysis. Let's imagine we want to explore records where carcinogenic chemicals that are listed as hazardous under the Clean Air Act were released on-site at facilities, either as "Fugitive air" or "Stack air" releases. This requires a complex combination of conditions, which we step through one by one below.

In [ ]:
# first, we can get all the records that released a Clean Air Act hazardous chemical.
searchFor = tri23ca['Clean air act chemical']=="YES" 
minitri23ca = tri23ca[searchFor]
# this yields 2123 records

# from those records that we know involve Clean Air Act chemicals, find the carcinogens.
searchFor = minitri23ca['Carcinogen']=="YES"
minitri23ca = minitri23ca[searchFor]
# this yields 729 records

# finally, let's select records where there was an air release. We'll keep the record in 
# the dataset if the amount of chemicals released as 'Fugitive air' or as 'Stack air'
# are larger than 0.
minitri23ca = minitri23ca[(minitri23ca['Fugitive air']>0) | (minitri23ca['Stack air']>0)]
minitri23ca # finally, there are a total of 554 records

# Wrangling Too Many Attributes <a class="anchor" id="attributes"></a>

Another common issue when using large public datasets for educational purposes is the problem of too many attributes. Environmental datasets such as the TRI22CA dataset we use here can include hundreds of specific indicators; survey datasets from organizations such as the Pew Research Center similarly report scores of questions per participant. While having access to so many attributes can enable the pursuit of a variety of investigative questions, it can easily become overwhelming. Working with these datasets requires planning and thoughtfulness to consider which attributes are actually connected to one's research question.

## Thematic Selection <a class="anchor" id="thematic"></a>

Thematic selection involves splitting a dataset up into related, but distinct, groups of attributes that are more likely to be conceptually or statistically related to one another. Thematic attribute selection can be especially useful for jigsaw-like activities, in which different groups explore different aspects of an interconnected system. Below, we create thematic groups by information type in each toxic release report. You can then access a dataset with a reduced number of selected attributes by calling the dataframe with the attribute group name in brackets. You can include attributes from multiple groups using the plus sign: `dataframe[selection1+selection2]`. Below, we create a dataset with only the facility information and basic release information.

In [ ]:
# Build our thematic categories using lists of column names
facility = ['Facility name','Federal facility']
location = ['Street address','City','County','St','Zip','Bia','Tribe','Latitude','Longitude','Horizontal datum']
company = ['Parent co name','Parent co db name','Standard parent co name','Foriegn parent co name','Foriegn parent co db num','Standard foriegn parent co name']
facilityType = ['Industry sector code','Industry sector','Primary sic','Sic 2','Sic 3','Sic 4','Sic 5','Sic 6','Primary naics','Naics 2','Naics 3','Naics 4','Naics 5','Naics 6']
filingInfo = ['Doc_ctrl_num','Form type']
basicInfo = ['Chemical','Elemental metal included','Clean air act chemical','Classification','Metal','Metal category','Carcinogen','PBT','PFAS','Unit of measure']
chemicalInfo = ['Tri chemical/compound id','Cas#','Srs id']
onSite = ['Fugitive air','Stack air','Water','Underground','Underground cl i','Underground cl ii-v','RCRA C landfill','Other landfills','Land treatment','RCRA Surface im','Other surface im','Other disposal','On-site total']
transfers = ['POTW Transfers for release','POTW Transfers for treatment','POTW Total transfers','Unclassified','Total transfer']
offSiteRelease = ['M10','M41','M62','M40 metal','M61 metal','M71','M81','M82','M72','M63','M66','M67','M64','M65','M73','M79','M90','M94','M99','Off-site release total']
offSiteRecycling = ['M20','M24','M26','M28','M93','Off-site recycled total']
offSiteRecovery = ['M56','M92','Off-site energy recovery t']
offSiteTreated = ['M40 non-metal','M50','M54','M61 non-metal','M69','M95','Off-site treated total']
toxicReleases = ['Total releases','Releases','On-site contained','Off-site other','Off-site contain','Off-site other r','Energy recover on','Energy recover of','Recycling on site','Recycling off sit','Treatment on site','Treatment off site','Production waste','One-time release','Prod_ratio_or_activity','Production ratio']

# use brackets to reference only the columns associated with one category.
minitri23ca = tri23ca[facility+basicInfo]
minitri23ca

## Pattern-Driven Selection <a class="anchor" id="pattern"></a>

Similar to thematic selection, pattern-driven selection allows educators and students to focus only on the attributes that are known to align with investigative or pedagogical goals. Whereas thematic selection focuses on the real-world relationships and system components that a student may wish to explore, pattern selection focuses attention on particular mathematical relationships and analytic methods that the dataset makes available for investigation.

Below, we demonstrate one example of pattern-driven selection that focuses on attribute type. We create a group focused on boolean variables about each toxic release event. Using this form of pattern-driven selection, educators and designers can introduce students to the methods and strategies for exploring categorical relationships in a dataset, and to identify the ones that might be especially worth pursuing further. We use Python to create crosstabs for a set of relevant columns in the dataframe. In BH100, we demonstrate how to similarly generate a correlation matrix to examine the patterns among quantitative columns in a dataframe.

In [ ]:
# Let's limit our dataset to some of the most relevant categorical attributes
toExplore = ['Federal facility',
             'Clean air act chemical',
             'Carcinogen',
             'PBT']
minitri23ca = tri23ca[toExplore]

# Create a dictionary to store crosstab matrices
crosstab_matrices = {}

# Generate crosstab matrices for each pair of columns
for col1 in toExplore:
    for col2 in toExplore:
        if col1 < col2:  # Just look pairwise at each column once
            crosstab_matrices[(col1, col2)] = pandas.crosstab(minitri23ca[col1], minitri23ca[col2], normalize='all').round(2)

# Print the crosstab matrices
for (col1, col2), matrix in crosstab_matrices.items():
    print(f"Crosstab: {col1} vs {col2}")
    print(matrix)
    print("-" * 30)

## Question-Driven Selection <a class="anchor" id="question"></a>

Question-driven selection involves identifying the attributes within a dataset that are most appropriate for addressing a particular question–whether that question is posed by a student, or posed as a driving question within curriculum materials themselves. We suggest making 3-5 attributes available for a given question in these curricular configurations. When students are constructing their own questions, we recommend encouraging them to also brainstorm what kind of data they would need to address their questions _before_ they have access to a dataset. Once they are provided with access, we suggest asking students to make predictions and construct hypotheses about what patterns they might find. 

Below, we assemble data to address the simple descriptive question, "What industries most frequently reported toxic release events in California in 2023?" We then dig deeper, supplementing the dataset to address a second question of whether those more frequently reporting industries are more likely than others to release carcinogens.

In [ ]:
exploreFreq = ['Facility name',
              'Industry sector'] 

minitri23ca = tri23ca[exploreFreq]

#count the number of records for each Industry sector category
industryCount = minitri23ca['Industry sector'].value_counts()
totalCount = sum(industryCount)

#show the record counts
print(industryCount)

#show the record counts as percent of total reports
industryCount/totalCount

We see that the two Industry sectors that most frequently report toxic release events are  "Chemicals" and "Petroleum." Together, these two sectors account for over one third of the toxic release reports in the dataset. We may wish to dig deeper, wondering whether these industries report releases of carcinogens more frequently than others. Let's see what proportion of top industries, versus other industries, report the release of carcinogenic chemicals.

In [ ]:
#let's add the carcinogen attribute to the dataset
minitri23ca = tri23ca[ exploreFreq + ['Carcinogen'] ]

#and, let's group industries by "top" and "other"
conditions = [minitri23ca['Industry sector'].isin(['Chemicals', 'Petroleum'])]
choices = ["YES"]
minitri23ca['Top Industry?'] = np.select(conditions, choices, default="NO")

#recode the 'Carcinogen' attribute with YES = 1 and NO = 0
conditions = [minitri23ca['Carcinogen'] == "YES"]
choices = [1]
minitri23ca['Carcinogen'] = np.select(conditions, choices, default=0)

#calculate difference in means between top and other industries
sum_by_category = minitri23ca.groupby('Top Industry?')['Carcinogen'].mean()
sum_by_category['YES'] - sum_by_category['NO']

This follow-up industry sector question motivates hypothesis generation. We can consider how our commonsense question translates into more formal hypotheses. The _null_ hypothesis is that the proportion of carcinogenic releases reported by top industries will be the same as the proportion of carcinogenic releases reported by other industries. While we can see above that the proportion is different--top industries report the release of carcinogenic toxins at a higher rate (32%) than other industries (25%), we need to check and see whether this difference in proportion is more likely due to existing variation in the dataset in general, rather than due to systematic differences between industry groups. 

We can create a simulation that generates what these data would look like if the industry category wasn't related to the likelihood of a reported carcinogenic release. We simulate this null hypothesis by randomly assigning different industries to each toxic release event, and recording the resulting difference in proportions of carcinogenic releases across those (randomly assigned) groups:

In [ ]:
# take all the Top Industry? values and shuffle them around
shuffle = minitri23ca.sample(frac=1,replace=False)['Top Industry?'].values

# add the shuffled values as a new column
minitri23ca['Shuffled Industry'] = shuffle

# see the proportion of carcinogenic events if they were assigned 
# to the new random, shuffled categories
sum_by_category = minitri23ca.groupby('Shuffled Industry')['Carcinogen'].mean()
abs(sum_by_category['YES'] - sum_by_category['NO'])

If we run the simulation many times, we get a distribution of means that we would expect from many, many instances of the null hypothesis in a probabilistic world.

In [ ]:
# create a function runIt to repeat the simulation process above
def runIt():
    shuffle = minitri23ca.sample(frac=1,replace=False)['Top Industry?'].values
    minitri23ca['Shuffled Industry'] = shuffle
    sum_by_category = minitri23ca.groupby('Shuffled Industry')['Carcinogen'].mean()
    return abs(sum_by_category['YES'] - sum_by_category['NO'])

distances = []

repetitions = 10000
for i in np.arange(repetitions):
    new_distance = runIt()
    distances = np.append(distances, new_distance)

sum_by_category = minitri23ca.groupby('Top Industry?')['Carcinogen'].mean()
observed = abs(sum_by_category['YES'] - sum_by_category['NO'])

print("DONE!")

After repeating this test 10000 times, we can see what the distribution of differences in proportions between the two top reporting industries ("Chemical" and "Petroleum") and compare the simulated distributions of the null hypothesis to our observed distribution.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(distances, bins=10)
plt.axvline(x=observed, color='magenta', label='Observed difference')

empirical_p = np.count_nonzero(distances >= observed) / repetitions
empirical_p

The purple line that represents the actual observed difference in proportions of carcinogenic releases by top reporters in the TRI23CA dataset versus other reporters is likely to appear even when there is no difference between the groups (in the simulation, the groups are randomly assigned to events). This is confirmed in our calculation of a rather large empirical p-value. This means we cannot reject the null hypothesis, that there is not a difference between the proportion of carcinogenic chemical releases reported by top industries versus other industries. We invite the ambitious reader, as an exercise, to revise the code above to examine the relationship between only the Petroleum industry and the release of carcinogenic chemicals.

# About the Toxic Release Inventory 2023 - CA Dataset 

This dataset includes all toxic release inventory reports recorded by the U. S. Environmental Protection Agency for facilitites in the State of California for the year 2023.

### About the Attribute Groups
Each record includes information about the reporting Facility (by federal identifiers), the Location of the release event (address, city, county, etc.), the Company responsible for the event, the Facility Type (e.g. industry categories) and Filing details (e.g. document tracking number and form submitted). Each toxic release record also includes Basic Information about the released chemicals and their risk factors, as well as more detailed Chemical Information, and there are several attribute groups with detailed reporting information depending on the type of release (e.g. whether the toxin was released on site, transferred elsewhere, recycled, treated, etc.).

### About the Attributes
Information about all 122 attributes can be found in the EPA's [Guide to the TRI Basic Data Files](https://www.epa.gov/system/files/documents/2024-08/basic_data_files_documentation_august_2024.pdf). They can also be reviewed by revieweing the metainformation available in the CODAP document [here](https://codap.concord.org/app/static/dg/en/cert/index.html#shared=https%3A%2F%2Fcfm-shared.concord.org%2FPvQBqkl1RCqPETiMddga%2Ffile.json). The attributes that are most frequently referenced in the Choosy paper include:
- *Facility Name* of the reporter.
- *City* in which the reporting facility is located.
- *Industry Sector* is a categorization of primary industry or sector the reporting facility belongs to.
- *Clean Air Act Chemical* is a boolean flag indicating whether the released chemical is listed as a hazardous air pollutant under the Clean Air Act.
- *Carcinogen* indicates whether the released chemical is identified as a carcinogen by the Occupational Health and Safety Administration.
- *PBT* indicates whether a chemical is identified as a persistent, bioaccumulative,
toxic (PBT) chemical.
- *PFAS* indicates whether a chemical is identified as a per- and polyfluoroalkyl substance (PFAS).

### History and Purpose
An earlier version of this dataset was initially imported into CODAP in Summer 2024 for consideration as part of the curricular materials developed for the Writing Data Stories project (DGE-2430522). It was updated in March 2025, to accompany "How to be Choosy," a manuscript submission about wrangling educational datasets to make them more managable and meaningful for classroom use.

### Data Sources and Data Cleaning
This dataset was downloaded on March 5, 2025 from the U. S. Environmental Protection Agency's [Toxic Release Inventory Basic Data Tool](https://www.epa.gov/toxics-release-inventory-tri-program/tri-basic-data-files-calendar-years-1987-present), and filtered to only show data for the state of California. Attributes were renamed for usability using the EPA's [Guide to the TRI Basic Data Files](https://www.epa.gov/system/files/documents/2024-08/basic_data_files_documentation_august_2024.pdf). Michelle Wilkerson of the WDS team  imported attribute titles and descriptions from the original dataset, editing a few descriptions for readability at the middle school level.

The development of these materials was supported by the National Science Foundation under Grant No. IIS-1900606.